# Interactive Plot_v2
Interactive plot to model Covid-19 in Indonesia. This repository is part of Covid-19 Modelling Project by [Department of Electrical and Information Engineering Universitas Gadjah Mada](http://jteti.ugm.ac.id/index.php?ver=YQ%3D%3D=). Interactive site using Voilà can be used using (need one to five minutes to load the plot):

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/yasirroni/Indonesia-Covid-Model/)

In [1]:
from IPython.display import display
from scipy.integrate import odeint
from datetime import datetime as dt

import ipywidgets as widgets
import numpy as np
import bqplot as bq
import pandas as pd
import itertools
import threading
import time

## Master setting
Edit this part if you want to run your own model. Otherwise, skip. For formating:  

1. compartments: `{'name1': [initial_value1], 'name2': [initial_value2], ...}`
2. variables: `{'name1': [value1, step1], 'name2': [value2, step2], ...}`
3. ode:
4. t: `['yyyy-mm-dd']`

In [2]:
master_compartments = widgets.Textarea(
    value = "{'S':[100], 'I':[1], 'R':[1], 'D':[1]}",
    placeholder = 'Type something',
    description = 'compartments:',
    disabled = False,
    layout = widgets.Layout(
        width = '100%',
    ),
    style = {
        'description_width': 'initial'
    }
)
display(master_compartments)

Textarea(value="{'S':[100], 'I':[1], 'R':[1], 'D':[1]}", description='compartments:', layout=Layout(width='100…

In [3]:
master_variables = widgets.Textarea(
    value = "{'beta':[100, 0.00001], 'gamma':[1, 0.00001], 'mu':[1, 0.00001]}",
    placeholder = 'Type something',
    description = 'variables :',
    disabled = False,
    layout = widgets.Layout(
        width = '100%',
    ),
    style = {
        'description_width': 'initial'
    }
)
display(master_variables)

Textarea(value="{'beta':[100, 0.00001], 'gamma':[1, 0.00001], 'mu':[1, 0.00001]}", description='variables :', …

In [4]:
master_t = widgets.Textarea(
    value = "['2020-01-31']",
    placeholder = 'Type something',
    description = 't:',
    disabled = False,
    layout = widgets.Layout(
        width = '100%',
    ),
    style = {
        'description_width': 'initial'
    }
)
display(master_t)

Textarea(value="['2020-01-31']", description='t:', layout=Layout(width='100%'), placeholder='Type something', …

## App

In [5]:
# input layout
layout_date = widgets.Layout(
    width = '250px'
) 

layout_inttext = widgets.Layout(
    width = '150px'
) 

# button layout
layout_button = widgets.Layout(
    width = '120px'
)

In [6]:
# init data to plot (dummy)
def init(t=['2020-01-01'], compartments = {'S':[100], 'I':[1], 'R':[1], 'D':[1]}):
    return [pd.Timestamp(t[0])], compartments
t, compartments = init(t=eval(master_t.value), compartments=eval(master_compartments.value))

In [7]:
def get_labels(compartments):
    return [i for i in compartments]

In [8]:
def get_y_lines(compartments):
    return [i for _,i in compartments.items()]

In [9]:
def get_labels_and_y_lines(compartments):
    labels = []
    y_lines = []
    for key, value in compartments.items():
        labels.append(key)
        y_lines.append(value)
    return labels, y_lines
labels, y_lines = get_labels_and_y_lines(compartments)

In [10]:
def get_color(number):
    '''Based on category20 in https://github.com/d3/d3-3.x-api-reference/blob/master/Ordinal-Scales.md#categorical-colors'''
    color_list = [
        '#1f77b4','#ff7f0e','#2ca02c','#d62728','#ff9896','#c5b0d5','#c49c94','#f7b6d2','#bcbd22','#17becf','#aec7e8','#ffbb78','#98df8a','#9467bd','#8c564b','#e377c2','#7f7f7f','#c7c7c7','#dbdb8d','#9edae5',
    ]
    
    if number < len(color_list):
        return color_list[0:number]
    else:
        quotient, remainder = divmod(number, len(color_list))
        quotient = color_list*quotient
        remainder = color_list[0:remainder]
        return [*quotient, *remainder]
colors = get_color(len(labels))

In [11]:
# layout_figure
layout_figure = widgets.Layout(width='500px', height='310px')

In [12]:
# output_widget
output_widget = widgets.Output()

In [13]:
# x_axis
x_sc = bq.DateScale()
x_ax = bq.Axis(
    label = 't',
    scale = x_sc
)

In [14]:
# y_lin
y_sc_lin = bq.LinearScale()
y_ax_lin = bq.Axis(
    label = 'y(t)',
    scale = y_sc_lin,
    orientation = 'vertical'
)
lines_lin = bq.Lines(
    x = t,
    y = y_lines,
    colors = colors,
    labels = labels,
    display_legend = True,
    scales = {
        'x': x_sc,'y': y_sc_lin
    }
)
scatter_lin = bq.Scatter(
    x = t*len(labels),
    y = list(itertools.chain.from_iterable(y_lines)),
    colors = ['#ff77ff00'],
    stroke_width = 1,
    display_legend = False,
    scales = {
        'x': x_sc,
        'y': y_sc_lin
    }, 
    tooltip = output_widget
)

In [15]:
fig_lin = bq.Figure(
    layout = layout_figure,
    axes = [x_ax, y_ax_lin],
    marks = [lines_lin, scatter_lin],
    fig_margin = dict(top=10, bottom=40, left=50, right=10),
    background_style = {'fill': 'white'}
)

In [16]:
# clear saved_points button
button_clear_saved_points = widgets.Button(
    description = 'CLEAR',
    button_style = '',
    layout = layout_button,
    disabled = True,
)

def button_clear_saved_points_click(_):
    global saved_points_list
    button_clear_saved_points.button_style = ''
    button_clear_saved_points.disabled = True
    saved_points_list = []
    saved_points.children = (*saved_points_list,)

button_clear_saved_points.on_click(button_clear_saved_points_click)

# saved_points
saved_points_list = []
saved_points = widgets.VBox(
    children = (*saved_points_list,)
)

# saved_points label
label_saved_points = widgets.Label(
    value = "Click plot to save point",
)

# saved_points wrapper
saved_points_wrapper = widgets.VBox(
    children = (label_saved_points, saved_points, button_clear_saved_points)
)

In [17]:
# box_plot
box_plot = widgets.HBox(
    children = (fig_lin,saved_points_wrapper)
)
display(box_plot)

In [18]:
def make_checkbox(labels = ['S', 'I', 'R', 'D']):
    checkboxes = []
    for label in labels:
        checkboxes.append(
            widgets.Checkbox(
                value=True,
                description=label,
                indent=False
            )
        )
    return tuple(checkboxes)

# checkbox_compartment
checkbox_compartment = make_checkbox(labels)

# checkbox_plot
checkbox_plot = make_checkbox(['Linear Scale', 'Log Scale'])

# checkboxes
checkboxes = widgets.HBox(
    children = (
        widgets.HBox(
            children = (*checkbox_compartment,)
        ),
        widgets.HBox(
            children = (*checkbox_plot,)
        )
    )
)
display(checkboxes)

In [19]:
# y_log
y_sc_log = bq.LogScale()
y_ax_log = bq.Axis(
    label = 'y(t)',
    scale = y_sc_log,
    orientation = 'vertical'
)
lines_log = bq.Lines(
    x = t,
    y = y_lines,
    colors = colors,
    labels = labels,
    display_legend = True,
    scales = {
        'x': x_sc,'y': y_sc_log
    }
)
scatter_log = bq.Scatter(
    x = t*len(labels),
    y = list(itertools.chain.from_iterable(y_lines)),
    colors = ['#ff77ff00'],
    stroke_width = 1,
    display_legend = False,
    scales = {
        'x': x_sc,'y': y_sc_log
    }, 
    tooltip = output_widget
)

In [20]:
fig_log = bq.Figure(
    layout = layout_figure,
    axes = [x_ax, y_ax_log],
    marks = [lines_log, scatter_log],
    fig_margin = dict(top=10, bottom=40, left=50, right=10),
    background_style = {'fill': 'white'}
)

In [21]:
# {'S':[100], 'I':[1], 'R':[1], 'D':[1]}

In [22]:
def make_inttext(compartments = {'S':[100], 'I':[1], 'R':[1], 'D':[1]}):
    inputs = []
    for key, value in compartments.items():
        inputs.append(
            widgets.IntText(
                value = value[0],
                description = key + ' initial:',
                layout = layout_inttext,
                style = {
                    'description_width': 'initial'
                }
            )
        )    
    return tuple(inputs)

In [23]:
# initial date
date_initial = widgets.DatePicker(
    description = 'Start Date:',
    disabled = False,
    value = t[0],
    layout = layout_date,
    style = {
        'description_width': 'initial'
    }
)

inttext_compartments = make_inttext(eval(master_compartments.value))

# checkboxes
initial = widgets.HBox(
    children = (date_initial, *inttext_compartments,)
)
display(initial)

In [24]:
initial.children[0]

DatePicker(value=Timestamp('2020-01-31 00:00:00'), description='Start Date:', layout=Layout(width='250px'), st…

In [25]:
# # initial button
# button_initial = widgets.Button(
#     description = 'ENTER',
#     button_style = 'primary',
#     layout = layout_button
# )

# def button_initial_click(_):
#     global flag_init_input
#     flag_init_input = True

# button_initial.on_click(button_initial_click)

# # initial wrapper
# initials = widgets.VBox(
#     children = (initial_date, S_init, I_init, R_init, D_init, button_initial),
# )

In [26]:
def make_floattext(variables = {'beta':[100, 0.00001], 'gamma':[1, 0.00001], 'mu':[1, 0.00001]}):
    inputs = []
    for key, value in variables.items():
        inputs.append(
            widgets.FloatText(
                value = value[0],
                step = value[1],
                description = key + ':',
                layout = layout_inttext,
                style = {
                    'description_width': 'initial'
                }
            )
        )    
    return tuple(inputs)

In [27]:
# checkpoint date
date_checkpoint = widgets.DatePicker(
    description = 'Checkpoint Date:',
    disabled = False,
    value = t[-1],
    layout = layout_date,
    style = {
    'description_width': 'initial'
    }
)

button_checkpoint = widgets.Button(
    description = 'Save Checkpoint',
    button_style = 'primary',
    layout = layout_button
)

def button_checkpoint_click(_):
    global flag_init_input
    flag_init_input = True

button_checkpoint.on_click(button_checkpoint_click)

floattext_variables = make_floattext(eval(master_variables.value))

# checkboxes
checkpoint = widgets.HBox(
    children = (date_checkpoint, *floattext_variables, button_checkpoint)
)
display(checkpoint)

In [28]:
# stop button
button_stop = widgets.Button(
    description = 'TURN OFF',
    icon = 'fa-toggle-on',
    button_style = 'success',
    layout = layout_button
)

def button_stop_click(_):
    global flag_on
    global thread
    if flag_on:
        flag_on = False
        button_stop.description = 'TURN ON'
        button_stop.icon = 'fa-toggle-off'
        button_stop.button_style = ''
    else:
        flag_on = True
        button_stop.description = 'TURN OFF'
        button_stop.icon = 'fa-toggle-on'
        button_stop.button_style = 'success'
        
        # create thread
        thread = threading.Thread(target=work)
        thread.name = 'work'
        
        # start thread
        thread.start()

button_stop.on_click(button_stop_click)

In [29]:
# line_hover
out_hover = widgets.Output(
    layout = widgets.Layout(
        margin = '0px 0px 0px 0px',
        padding = '0px 0px 0px 0px',
    )
)
def line_hover(_, hover_event):
    global out_hover
    out_hover.clear_output()
    with out_hover:
        print('Date:',dt.fromtimestamp(hover_event['data']['x']/1000).strftime('%d-%b-%Y'))
        print('Value:',int(hover_event['data']['y']))

In [30]:
# line_click
def line_click(_, click_event):
    global saved_points_list, t_list, labels
    button_clear_saved_points.disabled = False
    button_clear_saved_points.button_style = 'warning'
    label_index = click_event['data']['index']// len(t_list) 
    new_saved_point = widgets.Label(
        value = f"{labels[label_index]} Date: {dt.fromtimestamp(click_event['data']['x']/1000).strftime('%d-%b-%Y')} Value: {int(click_event['data']['y'])}"
    )
    saved_points_list.append(new_saved_point)
    saved_points.children = (*saved_points_list,)

In [31]:
scatter_lin.on_hover(line_hover)
scatter_lin.on_element_click(line_click)
scatter_log.on_hover(line_hover)
scatter_log.on_element_click(line_click)

In [32]:
# # app_wrapper
# app_input = widgets.VBox(
#     children = (initials,),
# #     layout = widgets.Layout(border = 'solid 2px gray', width='344px')
# )

# app_modifier = checkpoints
# app_screen = widgets.HBox(
#     children = (plots, saved_points_wrapper)
# )
# app_interactive = widgets.HBox(
#     children = (app_modifier, app_screen),
# #     layout = widgets.Layout(border = 'solid 2px gray', width='878px')
# )

In [33]:
def func_ode(y, t, beta, gamma, mu):
    [S, I, R, D] = y
    
    N = S + I + R + D
    
    dSdt = - (beta * S * I / N)
    dIdt = (beta * S * I / N) - ((gamma + mu) * I)
    dRdt = gamma*I
    dDdt = mu*I
    
    return [dSdt, dIdt, dRdt, dDdt]

In [34]:
def work():
    global flag_on, flag_init_input
    global t_list, S_list, I_list, R_list, D_list, t_list_str
    
    checkbox_log_prev = checkbox_log.value
    checkbox_lin_prev = checkbox_lin.value
    prev_val = []

    while flag_on:           
        # fig_log and fig_lin display
        if checkbox_log_prev != checkbox_log.value or checkbox_lin_prev != checkbox_lin.value:
            checkbox_log_prev = checkbox_log.value
            checkbox_lin_prev = checkbox_lin.value
            if checkbox_log.value and checkbox_lin.value:
                plots.children = (fig_lin, fig_log, checboxes)
            elif checkbox_log.value:
                plots.children = (fig_log, checboxes)
            elif checkbox_lin.value:
                plots.children = (fig_lin, checboxes)
            else:
                plots.children = (checboxes,)
        
#         # reset and init
#         if flag_init_input:           
#             init(str(initial_date.value), S_init.value, I_init.value, R_init.value, D_init.value)
#             t_start = initial_date.value
#             flag_init_input = False
        
        # main
        if prev_val != [
            initial_date.value, S_init.value, I_init.value, R_init.value, D_init.value,
            checkpoint_date.value, beta.value, gamma.value, mu.value, 
            checkbox_S.value, checkbox_I.value, checkbox_R.value, checkbox_D.value
            ]:
            
            prev_val = [
                initial_date.value, S_init.value, I_init.value, R_init.value, D_init.value,
                checkpoint_date.value, beta.value, gamma.value, mu.value, 
                checkbox_S.value, checkbox_I.value, checkbox_R.value, checkbox_D.value
            ]
            
            # make t
            t_len = int((checkpoint_date.value - initial_date.value).days) + 1 # the "+ 1" is to include endpoint
            t_new = pd.to_datetime(
                np.linspace(
                    pd.Timestamp(initial_date.value).value, 
                    pd.Timestamp(checkpoint_date.value).value, 
                    t_len,
                    endpoint=True
                )
            )       
            t_dummy = np.linspace(1, t_len, t_len)

            # make ode
            ode_out = odeint(
                func_ode,
                [S_init.value, I_init.value, R_init.value, D_init.value],
                t_dummy,
                args=(beta.value, gamma.value, mu.value)
            )

            # unpack ode_out
            S_new = ode_out[:,0]
            I_new = ode_out[:,1]
            R_new = ode_out[:,2]
            D_new = ode_out[:,3]

            # save checkpoint

            # merge and update plot
            t_list = t_new.tolist()
            S_list = S_new.tolist()
            I_list = I_new.tolist()
            R_list = R_new.tolist()
            D_list = D_new.tolist()
            t_list_str = [str(pd.Timestamp.date(i)) for i in t_list]               

            # update plot
            y = []
            y_scatter = []
            labels = []
            colors = []
            line_num = 0
            if checkbox_S.value:
                y.append(S_list)
                y_scatter.extend(S_list)
                labels.append(checkbox_S.description)
                colors.append('#2ca02c')
                line_num += 1
            if checkbox_I.value:
                y.append(I_list)
                y_scatter.extend(I_list)
                labels.append(checkbox_I.description)
                colors.append('#ff7f0e')
                line_num += 1
            if checkbox_R.value:
                y.append(R_list)
                y_scatter.extend(R_list)
                labels.append(checkbox_R.description)
                colors.append('#1f77b4')
                line_num += 1
            if checkbox_D.value:
                y.append(D_list)
                y_scatter.extend(D_list)
                labels.append(checkbox_D.description)
                colors.append('#d62728')
                line_num += 1

            if checkbox_lin:
                lines_lin.y = y
                lines_lin.labels = labels
                lines_lin.x = t_list
                lines_lin.colors = colors
                scatter_lin.x = t_list * line_num
                scatter_lin.y = y_scatter
                
            if checkbox_log:
                lines_log.y = y
                lines_log.labels = labels
                lines_log.x = t_list
                lines_log.colors = colors
                scatter_log.x = t_list * line_num
                scatter_log.y = y_scatter
        time.sleep(0.05)

In [35]:
# flag to control loop
# flag_on = True
# flag_init_input = False
# flag_add_checkpoint = False
# flag_compute = False

In [36]:
# set the flag to true
flag_on = True

# create thread
thread = threading.Thread(target=work)

# start thread
thread.start()

Exception in thread Thread-6:
Traceback (most recent call last):
  File "c:\users\muham\appdata\local\programs\python\python37\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "c:\users\muham\appdata\local\programs\python\python37\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-34-791a51ace2be>", line 5, in work
    checkbox_log_prev = checkbox_log.value
NameError: name 'checkbox_log' is not defined



In [37]:
# display app_input
display(app_input)

NameError: name 'app_input' is not defined

In [ ]:
# display app_interactive
display(app_interactive)

In [ ]:
button_stop